In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import subprocess
import sys
import os
import glob
import json
# try:
#   import open3d as o3d
# except:
#   subprocess.check_call([sys.executable, "-m", "pip", "install", "open3d"])
#   import open3d as o3d
import open3d as o3d
from config import *
from utils import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 1. Read Points Cloud

In [52]:
# read ply file
path = img_dir + "/2level_2.ply"
pcd = o3d.io.read_point_cloud(path)
# ply_file.paint_uniform_color([1, 0.706, 0])

# visualize ply file
# o3d.visualization.draw_geometries([pcd])


In [53]:
n_xyz = np.asarray(pcd.points)
print(n_xyz)
print("----------------------------")
n_xyz[:, 0] > 0

[[-2.28125   1.35645  -2.50977 ]
 [-2.27148   1.35742  -2.51172 ]
 [-2.26367   1.35938  -2.51563 ]
 ...
 [ 0.754395 -0.437988 -0.768555]
 [ 0.757813 -0.437988 -0.768555]
 [ 0.76123  -0.437988 -0.768555]]
----------------------------


array([False, False, False, ...,  True,  True,  True])

In [57]:
def crop(pcd, max_x, max_y, max_z):
  n_xyz = np.asarray(pcd.points)
  mask_x = n_xyz[:, 0] > max_x
  print(mask_x)
  mask_y = n_xyz[:, 1] > max_y
  mask_z = n_xyz[:, 2] > max_z
  mask = mask_x & mask_y & mask_z
  return n_xyz[mask]

max_x = -2
max_y = -1
max_z = -2

crop(pcd, max_x, max_y, max_z)

[False False False ...  True  True  True]


array([[ 1.10449 ,  0.943848, -1.74609 ],
       [ 1.10938 ,  0.941406, -1.74121 ],
       [ 1.11914 ,  0.942871, -1.74414 ],
       ...,
       [ 0.754395, -0.437988, -0.768555],
       [ 0.757813, -0.437988, -0.768555],
       [ 0.76123 , -0.437988, -0.768555]])

# 2. Detect distance

In [7]:
CUBOID_EXTENT_METERS = 200

METERS_BELOW_START = 5
METERS_ABOVE_START = 30

points = np.array([
    ## These points lie inside the cuboid
    [-2770.94365061042, 722.0595600050154, -20.004812609192445],
    [-2755.94365061042, 710.0595600050154, -20.004812609192445],
    [-2755.94365061042, 710.0595600050154, -15.004812609192445],

    ## These points lie outside the cuboid
    [-2755.94365061042 + CUBOID_EXTENT_METERS, 710.0595600050154, -15.004812609192445],
    [-2755.94365061042, 710.0595600050154 + CUBOID_EXTENT_METERS, -15.004812609192445],
  ]).reshape([-1, 3])

point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points)

## Start point here corresponds to an ego vehicle position start in a point cloud
start_position = {'x': -2755.94365061042, 'y': 722.0595600050154, 'z': -20.004812609192445}
cuboid_points = getCuboidPoints(start_position)

points = o3d.utility.Vector3dVector(cuboid_points)
oriented_bounding_box = o3d.geometry.OrientedBoundingBox.create_from_points(points)
point_cloud_crop = point_cloud.crop(oriented_bounding_box)

# View original point cloud with the cuboid, all 5 points present
o3d.visualization.draw_geometries([point_cloud, oriented_bounding_box])

# View cropped point cloud with the cuboid, only 3 points present
o3d.visualization.draw_geometries([point_cloud_crop, oriented_bounding_box])


In [29]:
a = np.random.random(size=(4, 3))
mask = (a[:,2] > 0.2 ) & (a[:, 1]> 0.2)
b = a[mask]
b

array([[0.71091805, 0.36624466, 0.5396815 ],
       [0.79690357, 0.48723825, 0.92339057],
       [0.41518979, 0.20390169, 0.41986006]])

# Read video

In [2]:
## License: Apache 2.0. See LICENSE file in root directory.
## Copyright(c) 2015-2017 Intel Corporation. All Rights Reserved.

###############################################
##      Open CV and Numpy integration        ##
###############################################

import pyrealsense2 as rs
import numpy as np
import cv2

from config import *

# Create Trackbar for Threshold
# cv2.namedWindow('Depth Image')
cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
cv2.createTrackbar('FPS', 'RealSense', 0, 50, lambda x: x+1)

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

fps = cv2.getTrackbarPos('FPS', 'RealSense')
print(fps)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, fps)

if device_product_line == 'L500':
    config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, fps)
else:
    config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, fps)

# Start streaming
pipeline.start(config)

try:
    while True:
        # print(pipeline)
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue
        
        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images = np.hstack((color_image, depth_colormap))
        
        # Flip camera image
        images = cv2.flip(images, 1)

        # Save images
        import os
        # Create directory if not exist
        if not os.path.exists('images'):
            os.mkdir('images')
        
        if count // 2 == 0:
            if len(os.listdir("./images"))!=10:
                cv2.imwrite("./images/image_" + str(count) + ".png", images)    
        count += 1

        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        key = cv2.waitKey(1)
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
        
finally:

    # Stop streaming
    pipeline.stop()

0
